In [1]:
import torch
from torch import nn
from timm import create_model
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, RandomHorizontalFlip, RandomVerticalFlip, RandomRotation
from tqdm import tqdm 
from torch.utils.data import ConcatDataset

train_dir='/kaggle/input/brain-tumor-classification-mri/Training'
test_dir='/kaggle/input/brain-tumor-classification-mri/Testing'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "efficientnet_b0"

classes=4


In [2]:
image_mean=[0.485, 0.456, 0.406]
image_std=[0.229, 0.224, 0.225]
transform = Compose([
    Resize((224, 224)),                              # Resize images
    ToTensor(),                                      # Convert image to tensor
    Normalize(mean=image_mean, std=image_std),  
])

transform1 = Compose([
    Resize((224, 224)), 
    RandomHorizontalFlip(p=1.0),
    ToTensor(),                                      
    Normalize(mean=image_mean, std=image_std),  
])
transform2 = Compose([
    Resize((224, 224)), 
    RandomVerticalFlip(p=1.0),
    ToTensor(),                                      
    Normalize(mean=image_mean, std=image_std),  
])
transform3 = Compose([
    Resize((224, 224)), 
    RandomHorizontalFlip(p=1.0),
    RandomVerticalFlip(p=1.0),
    ToTensor(),                                      
    Normalize(mean=image_mean, std=image_std),  
])


# Load dataset
train_dataset1 = ImageFolder(root=train_dir, transform=transform)
train_dataset2 = ImageFolder(root=train_dir, transform=transform1)
train_dataset3 = ImageFolder(root=train_dir, transform=transform2)
train_dataset4 = ImageFolder(root=train_dir, transform=transform3)

train_dataset =  ConcatDataset([train_dataset1, train_dataset2, train_dataset3, train_dataset4])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [3]:
model = create_model(model_name, pretrained=True, num_classes=classes)

# Optimizer and Loss Function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

model.to(device)

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2

In [4]:
def train(num_epochs):
    correct=0
    total=0
    running_loss = 0.0
    for epoch in range(num_epochs):  # Train for 10 epochs
        model.train()
        for batch in tqdm(train_loader):
            images, labels = batch
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            _, predictions = torch.max(outputs, 1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    
        print(f"Epoch {epoch + 1}: Train Accuracy = {correct / total:.2f}, Loss: {running_loss / len(train_loader)}")   
        
        
    

In [5]:
train(num_epochs=22)

100%|██████████| 359/359 [02:19<00:00,  2.58it/s]


Epoch 1: Train Accuracy = 0.89, Loss: 0.36247691747143573


100%|██████████| 359/359 [02:01<00:00,  2.95it/s]


Epoch 2: Train Accuracy = 0.94, Loss: 0.39436667934102254


100%|██████████| 359/359 [01:56<00:00,  3.08it/s]


...

Epoch 20: Train Accuracy = 0.99, Loss: 0.5777489885291377


100%|██████████| 359/359 [01:53<00:00,  3.17it/s]


Epoch 21: Train Accuracy = 0.99, Loss: 0.5834088407798658


100%|██████████| 359/359 [01:52<00:00,  3.20it/s]

Epoch 22: Train Accuracy = 0.99, Loss: 0.5852975205343102


In [6]:
#test
def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_loader:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _,predictions = torch.max(outputs, 1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    print(f"Test Accuracy = {correct / total:.2f}")

test_dataset = ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
test()

Test Accuracy = 0.79
